In [9]:
import names
import re
import functions.markovgen as markovgen
import random
import uuid

from datetime import datetime
from time import time
from functions.get_book import *

In [15]:
def random_date(timex=False):
    random.seed(random.random())
    d = random.randint(1, int(time()))

    if timex:
        return datetime.fromtimestamp(d).strftime('%Y-%m-%d %H:%M:%S')
    return datetime.fromtimestamp(d).strftime('%Y-%m-%d')

# Users

In [76]:
regex = re.compile('[^a-zA-Z ]')

file_ = open('text.txt')
results_ = open('./mocks/users.txt', 'w')

markov = markovgen.Markov(file_)

for i in range(4, 500):
    name = names.get_full_name()

    person = [i,
              name.lower().replace(' ', '.') + '@teste.com',
              name,
              name.split(' ')[0],
              regex.sub('', markov.generate_markov_text(5).capitalize()),
              '$2b$12$oVdwTmriZq4StlbbFXIrlO9WxXbwg5TRnv6xIiRBh7OQfdnTb12nu',
              random_date(),
              'https://clipground.com/images/clipart-profile-6.jpg',
              True,
              str(uuid.uuid4())]

    results_.write('(')
    results_.write(',\n'.join([f"'{w}'"for w in person]))
    results_.write('), \n')

file_.close()
results_.close()

# Books

In [11]:
words = ('cara', 'menina', 'menino', 'dragão', 'bruxa', 'água', 'tempo', 'chuva', 'terra', 'choro', 'brasil', 'pedra', 'nome')

aux, id, i = 0, 1, 0

results_ = open('./mocks/books.txt', 'w')

while id < 2000:
    books = get_by_term(words[aux], i)
    try:
        for book in books['items']:
            results_.write(f"({id}, '{book['id']}'),\n")
            id += 1
    except:
        aux += 1
        i = 0

results_.close()

# User Books

In [23]:
id = 1

results_ = open('./mocks/user_book.txt', 'w')
all_combinations = []
while id < 1500:
    combination = [random.randint(1, 2000), random.randint(1, 499)]
    if combination not in all_combinations:
        results_.write(f"({combination[0]}, {combination[1]}, {random.randint(1, 3)}, '{random_date(True)}'),\n")
        id += 1
        all_combinations.append(combination)

results_.close()

# User Genre

In [25]:
id = 0

results_ = open('./mocks/user_genre.txt', 'w')
all_combinations = []
while id < 1200:
    combination = [random.randint(1, 10), random.randint(1, 499)]
    if combination not in all_combinations:
        results_.write(f"({combination[0]}, {combination[1]}),\n")
        id += 1
        all_combinations.append(combination)
results_.close()

# Friends

In [41]:
id = 0

results_ = open('./mocks/friend.txt', 'w')
all_combinations = []
while id < 1500:
    combination = [random.randint(1, 499), random.randint(1, 499)]
    if combination not in all_combinations \
            and combination[0] != combination[1]\
            and combination[::-1] not in all_combinations:
        results_.write(f"({bool(random.getrandbits(1))}, "
                       f"{bool(random.getrandbits(1))}, "
                       f"{combination[0]}, "
                       f"{combination[1]}),\n")
        id += 1
        all_combinations.append(combination)

results_.close()

[204, 1]